In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
driver = webdriver.Chrome(chromedriver)

In [2]:
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
from statistics import mean

In [4]:
import pandas as pd

In [5]:
height_to_inches = {"5'5":65,
"5'6":66,
"5'7":67,
"5'8":68,
"5'9":69,
"5'10":70,
"5'11":71,
"6'0":72, 
"6'1":73,
"6'2":74,
"6'3":75,
"6'4":76,
"6'5":77,
"6'6":78,
"6'7":79,
"6'8":80,
"6'9":81,
"6'10":82,
"6'11":83}

In [6]:
URL = "https://www.baseball-reference.com/teams/CHW/2000-roster.shtml"

In [5]:
driver.get(URL)

In [6]:
roster_table = driver.find_element_by_xpath('//table[@id="appearances"]')
roster_lines = roster_table.find_elements_by_css_selector('tr')

In [7]:
for item in roster_lines[:5]:
    print(item.text)
    print("-----")

Name Age B T Ht Wt DoB Yrs G GS Batting Defense P C 1B 2B 3B SS LF CF RF OF DH PH PR WAR Salary
-----
Jeff Abbott 27 us US R L 6' 2" 190 Aug 17, 1972 4 80 52 80 65 0 0 0 0 0 0 20 33 16 65 7 18 2 -0.3 $255,000
-----
Harold Baines HOF 41 us US L L 6' 2" 175 Mar 15, 1959 21 24 15 24 0 0 0 0 0 0 0 0 0 0 0 16 9 0 -0.3
-----
James Baldwin 28 us US R R 6' 3" 210 Jul 15, 1971 6 29 28 2 29 29 0 0 0 0 0 0 0 0 0 0 0 0 2.7 $3,300,000 All-Star
-----
Lorenzo Barcelo 22 do DO R R 6' 4" 230 Aug 10, 1977 1st 22 1 0 22 22 0 0 0 0 0 0 0 0 0 0 0 0 0.8 $200,000
-----


In [16]:
last_string = roster_lines[-1].text.split(" ")
print(last_string)

['Name', 'Age', 'B', 'T', 'Ht', 'Wt', 'DoB', 'Yrs', 'G', 'GS', 'Batting', 'Defense', 'P', 'C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF', 'OF', 'PH', 'PR', 'WAR', 'Salary']


In [11]:
string_1 = roster_lines[1].text.split(" ")

In [16]:
print("Weight is",string_1[9])
print("Height is",string_1[7]," feet, ",string_1[8]," inches.")
print("Number of games pitched ",string_1[18])

Weight is 190
Height is 6'  feet,  2"  inches.
Number of games pitched  0


In [22]:
height = string_1[7] + string_1[8]
print(height)

6'2"


In [14]:
string_2 = roster_lines[2].text.split(" ")

In [18]:
print("Weight is",string_2[10])
print("Height is",string_2[8]," feet, ",string_1[9]," inches.")
print("Number of games pitched ",string_2[19])

Weight is 175
Height is 6'  feet,  190  inches.
Number of games pitched  0


In [19]:
string_3 = roster_lines[3].text.split(" ")

In [20]:
print("Weight is",string_3[9])
print("Height is",string_3[7]," feet, ",string_1[8]," inches.")
print("Number of games pitched ",string_3[18])

Weight is 210
Height is 6'  feet,  2"  inches.
Number of games pitched  29


In [6]:
df = pd.read_csv("baseball_output7.csv",converters={"Year":str})

In [12]:
count = 1
while count < 2:    
    #CHANGE DRIVER TO ROSTER
    #Batting & pitching weights & heights
    driver.get(df["Roster_URL"][count])
    batting_height_list = []
    pitching_height_list = []
    batting_weight_list = []
    pitching_weight_list = []
    new_batting_height_list = []
    new_pitching_height_list = []
    roster_table = driver.find_element_by_xpath('//table[@id="appearances"]')
    WebDriverWait(driver,3)
    roster_lines = roster_table.find_elements_by_css_selector('tr')
    i = 1
    while i < len(roster_lines)-1:
        string_row = roster_lines[i].text.split(" ")
        age = string_row[2]
        if age.isdigit() == True:
            weight = string_row[9]
            height = str(string_row[7]+string_row[8]).replace("''","").replace("""""","").replace("\"","")
            games_pitched = string_row[18]
        #Account for players with "HOF", which shifts the data left
        else:
            weight = string_row[10]
            height = str(string_row[8] + string_row[9]).replace("''","").replace("""""","").replace("\"","")
            games_pitched = string_row[19]
        if int(games_pitched)>0:
            pitching_height_list.append(height)
            pitching_weight_list.append(weight)
            i+=1
        else:
            batting_height_list.append(height)
            batting_weight_list.append(weight)
            i+=1
    new_batting_height_list = [height_to_inches[x] for x in batting_height_list]
    new_pitching_height_list = [height_to_inches[x] for x in pitching_height_list]
    new_batting_weight_list = [int(x) for x in batting_weight_list]
    new_pitching_weight_list = [int(x) for x in pitching_weight_list]
    df["Average_Height_of_Pitcher"][count]=mean(new_pitching_height_list)
    df["Average_Height_of_Batter"][count]=mean(new_batting_height_list)
    df["Average_Weight_of_Pitcher"][count]=mean(new_pitching_weight_list)
    df["Average_Weight_of_Batter"][count]=mean(new_batting_weight_list)
    WebDriverWait(driver,3)
    count+=1

/var/folders/ny/g82ykkn10bd8fqbx1q14fk_m0000gn/T/ipykernel_54215/376720247.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Average_Height_of_Pitcher"][count]=mean(new_pitching_height_list)
/var/folders/ny/g82ykkn10bd8fqbx1q14fk_m0000gn/T/ipykernel_54215/376720247.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Average_Height_of_Batter"][count]=mean(new_batting_height_list)
/var/folders/ny/g82ykkn10bd8fqbx1q14fk_m0000gn/T/ipykernel_54215/376720247.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [15]:
df.head()

,Unnamed: 0,Year,Team,Overall TEAM,Wins,Losses,Win-Loss_%,Total_Batting_Salary,Average_Batting_Age,Average_Years_Batting,Total_Pitching_Salary,Average_Pitching_Age,Average_Years_Pitching,Average_Height_of_Pitcher,Average_Weight_of_Pitcher,Average_Height_of_Batter,Average_Weight_of_Batter,Batting_URL,Pitching_URL,Roster_URL
0,0,1980,ATL,ATL,81,80,1.012500,2071667,26.7,6.125000,1706667.0,29.8,6.727273,NaN,NaN,NaN,NaN,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...
1,1,1981,ATL,ATL,50,56,0.892857,5571200,26.3,5.400000,3185367.0,31.9,6.687500,72.9375,186.25,71.916667,179.125,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...
2,2,1982,ATL,ATL,89,73,1.219178,2745166,26.9,5.358974,2026666.0,29.7,6.000000,NaN,NaN,NaN,NaN,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...
3,3,1983,ATL,ATL,88,74,1.189189,5008233,27.6,6.150000,1438333.0,29.2,6.470588,NaN,NaN,NaN,NaN,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...
4,4,1984,ATL,ATL,80,82,0.975610,5533333,27.7,6.710526,1383333.0,28.4,6.133333,NaN,NaN,NaN,NaN,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...,https://www.baseball-reference.com/teams/ATL/1...


In [ ]:
df.to_csv('baseball_output8.csv')

1:54 is the last run time